# The Dreambooth Ion Cannon

    The first thing I need to say is that this is not original work.
    
    It is based off of "The Repo Formerly Known As Dreambooth" by Joe Penna and his many extraordinarily talented collaborators, and further tweaks to the training process have been made by my friend Kane Wallmann. I am in awe of them, and owe them all a bottle of something from the top shelf.
    
    Dreambooth is a phenomenal piece of work, despite being described as 'walking through a jungle without a torch'.
    
    This variant of the repository is not an "improvement" per se - rather, it is a simplification of the process of
    image generation by virtue of abstracting away details such as requiring you to interface with external sites, and
    providing some prompts that I've found work quite well across the board.
    
    The cost to this abstraction is a significantly larger base repository, coming out at 4.8 gigabytes. Them's the breaks.
    
    There is still a level of technical know-how required, however you - the intended audience - are unlikely to be
    reading this unless you've already followed part of the guide in the README for this repository fork, so you're
    already most of the way there.
    
    Some provisos:
        * I have bundled Stable Diffusion 1.4 in the repository containing this notebook. Please read the license 
          file `STABLE DIFFUSION LICENSE.md` in order to understand what you implicitly agree to by making use of this tech.
          The long and short of it is that so long as you don't approach this tool with ill intent, you're fine.
        * Despite Kane's adjustments enabling multi-subject training, this repo *at present* is designed to only accept 
          one subject at a time - that's what the prompts are expecting, so training multiple people is a waste. In future
          iterations of this repository, I will introduce some branching logic allowing you to select how many people you
          want to include in the generated images.
          
    I hope that that this inspires a little bit of wonder in you.
    
    /Laurence

# Naming And Classification

\*\*\* This is the **only** cell you have to edit. \*\*\* 

Before you opened this notebook per instructions from the repository, you should have uploaded 8-10 .png photographs of the person you are trying to generate images for into the directory `/training_samples/{class}` with filenames:

* `name class_001.png`
* `name class_001.png`
* ...
* `name class_00n.png`

If you haven't done this, go back to the README of this repository (scroll down from the link you were given) and give it a go.

Your choices for class are `man`, `woman`, `person`, `dog` or `cat`, whereas name is dealers choice, but do keep it as a single word (i.e. no spaces).

**For example**, if you're trying to train on Princess Diana, your files would be called `princessdiana woman_001.png` et cetera.

**It is advisable to use a name choice so that - if you're called Willow, for example - the model doesn't think you're Willow from Buffy, or think you're Asian if your name is Jian. I recommend using a name such as `jianthisisdefinitelyme` if you have a name that could cause confusion.**

The spacing and underscores are important here, so please double-check.

In the cell below, edit the variables to reflect the `name` and `class` you have chosen. _Again_, please check for accuracy.

In [ ]:
target_name = "princessdiana"

target_class = "woman"

### If you do not want to adjust any of the optional parameters (see below)...

# You can now click *Cell > Run All* from the menu bar above.

## Go away for 90 minutes.

---

## Optional Parameters

The `run_length` variable dictates how many images are generated for each prompt.

The default, `medium`, produces 64 images per prompt, but should you choose, you can edit this to either:

* `short` - _25_ images per prompt
* `long` - _100_ images per prompt

Note that shortening the run length will produce results faster, but give you fewer results from which to select a true likeness, whereas lengthening it will slow things down by a fair margin, but produce a much wider choice at the end.

----

The `spicy_mode` variable determines whether the Ion Cannon will generate images from some of the bolder prompts for you.

Think burlesque dancers - as a rule it takes a more laissez-faire approach to clothing and sensuality, although none of the prompts direct full-blown nudity.

This mode is off by default, and can be activated by changing `False` to `True` (no quotes).

I would gently ask - although I obviously cannot enforce this - that if you are using the Ion Cannon on people that are not yourself, you request permission to do this, as doing otherwise has the potential to lead to a breach of the TOS of Stable Diffusion that you agree to by utilising this code.

It produces cosmic horrors if you use it on your dog, though.

In [ ]:
run_length = "medium"

spicy_mode = False

# Build Environment

In [ ]:
#BUILD ENV
!pip install omegaconf
!pip install einops
!pip install pytorch-lightning==1.6.5
!pip install test-tube
!pip install transformers
!pip install kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install setuptools==59.5.0
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0
!pip install -e .
!pip install protobuf==3.20.1
!pip install gdown
!pip install pydrive
!pip install -qq diffusers["training"]==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"
!pip install huggingface_hub
!pip install ipywidgets==7.7.1
!pip install py7zr==0.20.0
!pip install multivolumefile

# Decompressing/Reconstituting Stable Diffusion v1.4

In [ ]:
import multivolumefile
import py7zr
with multivolumefile.open('/workspace/Dreambooth-Ion-Cannon/base_model/model.7z', mode='rb') as target_archive:
    with py7zr.SevenZipFile(target_archive, 'r') as archive:
        archive.extractall()

# Retrieving Regularisation Images

In [ ]:
!git clone https://github.com/laurenceday/Ion-Cannon-Regularisation-{target_class}.git

!mkdir -p regularisation_images/{target_class}
!mv -v Ion-Cannon-Regularisation-{target_class}/{target_class}/*.* regularisation_images/{target_class}

# Training and Pruning

In [ ]:
target_token = f"{target_name} {target_class}"
project_name = f"{target_name}_model"
model_name = f"{project_name}.ckpt"

In [ ]:
!rm -rf training_samples/{target_class}/.ipynb_checkpoints
!python "main.py" \
 --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
 -t \
 --actual_resume "model.ckpt" \
 --reg_data_root "/workspace/Dreambooth-Ion-Cannon/regularisation_images" \
 -n {project_name} \
 --gpus 0, \
 --data_root "/workspace/Dreambooth-Ion-Cannon/training_samples" \
 --max_training_steps 1000 \
 --no-test

In [ ]:
directory_paths = !ls -d logs/*

In [ ]:
# This version should automatically prune around 10GB from the ckpt file
last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"
!python "prune_ckpt.py" --ckpt {last_checkpoint_file}

In [ ]:
last_checkpoint_file_pruned = directory_paths[-1] + "/checkpoints/last-pruned.ckpt"
!mkdir -p trained_models
!mv {last_checkpoint_file_pruned} trained_models/{model_name}

In [ ]:
model_path = f"/workspace/Dreambooth-Ion-Cannon/trained_models/{model_name}"

# Image Generation Variable Setup

In [ ]:
if (run_length == "short"):
    sample_n = 5
elif (run_length == "long"):
    sample_n = 10
else:
    sample_n = 8

# Regular Image Generation

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 2 \
 --n_iter 2 \
 --seed 42 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "{target_token}" \
 --outloc "{target_name}_raw" \
 --outdir "outputs/{target_name}_raw"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples {sample_n} \
 --n_iter {sample_n} \
 --seed 2603348708 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "A beautiful portrait of a {target_token} with diamonds and glittering skin, a detailed painting by greg rutkowski and raymond swanland, featured on cgsociety, fantasy art, detailed painting, artstation hd, photorealistic, fantasy, intricate, elegant, rainbow bubbles, highly detailed, digital painting" \
 --outloc "{target_name}_ethereal" \
 --outdir "outputs/{target_name}_ethereal"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples {sample_n} \
 --n_iter {sample_n} \
 --seed 4085433209 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "Up close portrait of a beautiful {target_token} in black and white, photorealistic, upper body, art by diego fazio and diegokoi and artgerm, concept art, hyper sharp focus, 8k highly detailed" \
 --outloc "{target_name}_blackwhite" \
 --outdir "outputs/{target_name}_blackwhite"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples {sample_n} \
 --n_iter {sample_n} \
 --seed 69 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "digital painting of {target_token} by greg rutkowski neon cyberpunk" \
 --outloc "{target_name}_cyberpunk" \
 --outdir "outputs/{target_name}_cyberpunk"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples {sample_n} \
 --n_iter {sample_n} \
 --seed 1201562208 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "A soft and breathtaking detailed painting of a {target_token} with a crown on their head in the style of gustav klimt, shiny gold, elegant, highly detailed, artstation, fluo colors, concept art, matte, sharp focus, art by gustav klimt and alphonse mucha" \
 --outloc "{target_name}_crown" \
 --outdir "outputs/{target_name}_crown"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples {sample_n} \
 --n_iter {sample_n} \
 --seed 3491584590 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "bioshock {target_token} as big sister portrait, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, art by artgerm and greg rutkowski and alphonse mucha" \
 --outloc "{target_name}_bioshock" \
 --outdir "outputs/{target_name}_bioshock"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples {sample_n} \
 --n_iter {sample_n} \
 --seed 69 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "portrait of {target_token} cyberpunk neon - bordered cyborg, 7 0 mm focal length, by ilya kuvshinov, krenz cushart, Greg Rutkowski, trending on artstation sharp focus illustration, aesthetic, very inspirational, arthouse" \
 --outloc "{target_name}_cyborg" \
 --outdir "outputs/{target_name}_cyborg"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples {sample_n} \
 --n_iter {sample_n} \
 --seed 1337 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "digital painting of {target_token} cosmic background, scenery landscape, professional, award - winning, trending on artstation, hyper detailed, realistic, beautiful, emotional, shiny, golden, picture" \
 --outloc "{target_name}_cosmic" \
 --outdir "outputs/{target_name}_cosmic"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples {sample_n} \
 --n_iter {sample_n} \
 --seed 1337 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "{target_token} cosmic background realistic shaded perfect face, fine details by realistic shaded lighting poster by ilya kuvshinov katsuhiro otomo, magali villeneuve, artgerm, jeremy lipkin and michael garmash and rob rey" \
 --outloc "{target_name}_cosmic2" \
 --outdir "outputs/{target_name}_cosmic2"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples {sample_n} \
 --n_iter {sample_n} \
 --seed 1694397999 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "a beautiful portrait of {target_token}, with isolated flowers with strong dark comic outlines, colorful, psychedelic, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, art by artgerm and greg rutkowski and alphonse mucha" \
 --outloc "{target_name}_flowers" \
 --outdir "outputs/{target_name}_flowers"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples {sample_n} \
 --n_iter {sample_n} \
 --seed 2148934030 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "{target_token}, portrait shinkai makoto studio ghibli studio key hideaki anno sakimichan stanley artgerm lau rossdraws james jean marc simonetti elegant highly detailed digital painting artstation pixiv" \
 --outloc "{target_name}_ghibli" \
 --outdir "outputs/{target_name}_ghibli"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples {sample_n} \
 --n_iter {sample_n} \
 --seed 2283716726 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "Portrait of a beautiful {target_token}, intricate, elegant, highly detailed, occult imagery, digital painting, artstation, concept art, illustration, by gil elvgen, greg manchess, mucha" \
 --outloc "{target_name}_paint" \
 --outdir "outputs/{target_name}_paint"

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples {sample_n} \
 --n_iter {sample_n} \
 --seed 763931883 \
 --scale 10.0 \
 --ddim_steps 100 \
 --ckpt {model_path} \
 --prompt "Duotone trippy 1 9 6 0 s lsd concept illustration of {target_token}. volumetric lighting. golden ratio accidental renaissance. by sachin teng and sergey kolesov and ruan jia and heng z. graffiti art, scifi, fantasy, hyper detailed. octane render. concept art. trending on artstation." \
 --outloc "{target_name}_trippy" \
 --outdir "outputs/{target_name}_trippy"

# Spicy Mode Image Generation

More prompts coming soon, this is just a stand-in I find works quite well.

In [ ]:
if (spicy_mode):
    !python scripts/stable_txt2img.py \
     --ddim_eta 0.0 \
     --n_samples {sample_n} \
     --n_iter {sample_n} \
     --seed 3369866510 \
     --scale 10.0 \
     --ddim_steps 100 \
     --ckpt {model_path} \
     --prompt "A beautiful {target_name} burlesque dancer on a stage, spotlight, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, cinematic lighting, art by artgerm and greg rutkowski and alphonse mucha" \
     --outloc "{target_name}_burlesque" \
     --outdir "outputs/{target_name}_burlesque"